# Demo Notebook for the Orca Detector Project
#### Spyros Garyfallos, Mike Winton, Ram Iyer

### Import Dependencies

In [1]:
import glob
import os
import pandas as pd
import shutil
import datetime
import time
import numpy as np

import random
import IPython.display as ipd
import urllib

import  orca_params
from live_feed_listener import perform_inference
from inference import create_network

from scipy.io import wavfile
import ipywidgets as widgets
from IPython.display import display

import shutil

import matplotlib.pyplot as plt
%matplotlib inline

import librosa
import librosa.display
import mel_params

import demo_helper

Using TensorFlow backend.


## Instantiate object to process input data from raw audio files and convert to dataframe

In [2]:
from IPython.utils import io
with io.capture_output() as captured:
    mammals = demo_helper.MammalFind()

### Filter out labels which have very few samples

In [3]:
valid_labels = mammals.get_valid_labels()

In [4]:
mel_utils = demo_helper.MelUtils()

### Code to run Inference

In [5]:
#from live_feed_listener import perform_inference
weights_path = '/results/vggish/weights.best.hdf5'
label_encoder_path = '/results/label_encoder_latest.p'
probability_threshold = orca_params.LIVE_FEED_MINIMUM_INFERENCE_PROBABILITY
model_name = orca_params.DEFAULT_MODEL_NAME
inference_samples_path = "./inference_output"

model, encoder = create_network(
        model_name, label_encoder_path, weights_path)

def display_inference_results(audio_source):
    
    if audio_source == "mammal":
        inference_samples_path = "./mammal_inference_path"
    elif audio_source == "noise":
        inference_samples_path = "./noise_inference_path"
    else:
        inference_samples_path = "./inference_output"
        
    start_timestamp = datetime.datetime.now()
    perform_inference(model,encoder, inference_samples_path, probability_threshold)

    f = widgets.IntProgress(description = "Processing Results", bar_style="info",min=0, max=3)
    display(f)
    for i in range(4):
        f.value = i
        time.sleep(0.5)

    list_of_files = glob.glob('/results/detections/*/*/*.csv')
    if len(list_of_files) == 0:
        print("No species found")
        return
    latest_file = max(list_of_files, key=os.path.getctime)

    file_timestamp = datetime.datetime.fromtimestamp(os.path.getctime(latest_file))
    print("start time {}, file time{}".format(start_timestamp.isoformat('-'), file_timestamp.isoformat('-')))
    if (file_timestamp <= start_timestamp):
        print("No Species Detected")
        return

    results_df = pd.read_csv(latest_file)
    results_df = results_df[['0','1','2']]
    results_df.columns = ['FileName','Species','Probability']
    display(results_df)
    species_detected = results_df.loc[results_df['Probability'].idxmax(),'Species']
    print("Dominant Species detected is {}".format(species_detected))
        
    return
    

Loading trained LabelEncoder from /results/label_encoder_latest.p
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loading weights from /results/vggish/weights.best.hdf5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 64, 1)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 64, 1)         4         
_________________________________________________________________
conv1 (Conv2D)               (None, 96, 64, 64)        640       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 48, 32, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (No

### Define Utility functions for playing Audio

In [6]:
def play_noise_sample(stream_name,volume):
    file = demo_helper.get_noise_sample(stream_name,volume)
    ipd.display(ipd.Audio(file))
    return file

In [7]:
def play_combined_sample(mammal_name, mammal_volume,
                         noise_stream_name,noise_stream_volume):

    file = demo_helper.get_combined_sample(mammal_name,mammal_volume,
                                          noise_stream_name,noise_stream_volume)
    
    ipd.display(ipd.Audio(file))     
   
    return file

### Define Widgets for the user interface

In [8]:
mammal_play_time = 10
mammal_plt_output = widgets.Output()
noise_plt_output = widgets.Output()
mammal_audio_output = widgets.Output()
noise_audio_output = widgets.Output()
combined_audio_output = widgets.Output()
combined_plt_output = widgets.Output()
combined_inference_output = widgets.Output()

In [9]:
stream_bases = orca_params.ORCASOUND_STREAMS.keys()

mammal_species = widgets.Dropdown(options=sorted(valid_labels), value="KillerWhale")
mammal_volume = widgets.FloatSlider(value=1.0,min=0, max=3, step=0.01, 
                                    description="Volume",
                                   continuous_update=False)

noise_stream = widgets.Dropdown(options=stream_bases,value="OrcasoundLab")
noise_volume = widgets.FloatSlider(value=0.05,min=0, max=1, step=0.005, 
                                    description="Volume",
                                   continuous_update=False)

inf_required = widgets.Checkbox(value=False, description = 'Run Inference', disabled=False)

### Define callback functions

In [10]:
def combined_processing(mammal_species, mammal_volume,
                       noise_source, noise_volume):
    
    #print ("Mammal Species {}".format(mammal_species))
    #print("Noise source {}".format(noise_source) )
    combined_plt_output.clear_output()
    combined_audio_output.clear_output()
    inf_required.value = False
    
    file_name = None
    with combined_audio_output:
        print("Combined Audio")
        file_name = play_combined_sample(mammal_species, mammal_volume,
                                        noise_source, noise_volume)
        mel_utils.display_wave(file_name)
    with combined_plt_output:
        print("Combined Mel Spectrogram")
        mel_utils.display_mel(file_name)

def mammal_processing(species,volume):
    mammal_plt_output.clear_output()
    mammal_audio_output.clear_output()
    inf_required.value = False
    
    with mammal_audio_output:
        s_file = mammals.get_sample_sound(species,volume,mammal_play_time)
        print(s_file)
        ipd.display(ipd.Audio(s_file))
        mel_utils.display_wave("./display_sounds/output.wav")
    with mammal_plt_output:
        mel_utils.display_mel("./display_sounds/output.wav")
        
def noise_processing(noise_source,volume):
    noise_audio_output.clear_output()
    noise_plt_output.clear_output()
    inf_required.value = False
    
    file_name = None
    with noise_audio_output:
        file_name = play_noise_sample(noise_source,volume)
        mel_utils.display_wave(file_name)
    with noise_plt_output:
        mel_utils.display_mel(file_name)
        

### Define Event Handlers

In [11]:
def mammal_species_eventhandler(change):
    new_species = mammals.get_sample(change.new)
    mammal_processing(new_species,mammal_volume.value)
    combined_processing(new_species,mammal_volume.value,
                       noise_stream.value, noise_volume.value)
    
def mammal_volume_eventhandler(change):
    species = mammals.get_sample(mammal_species.value)
    mammal_processing(species, change.new)
    combined_processing(species,change.new,
                       noise_stream.value, noise_volume.value)

def noise_stream_eventhandler(change):
    species = mammals.get_sample(mammal_species.value)
    noise_processing(change.new,noise_volume.value)
    combined_processing(species,mammal_volume.value,
                       change.new,noise_volume.value)
    
def noise_volume_eventhandler(change):
    species = mammals.get_sample(mammal_species.value)
    noise_processing(noise_stream.value, change.new)
    combined_processing(species,mammal_volume.value,
                        noise_stream.value,change.new)
    
def inference_eventhandler(change):
    combined_inference_output.clear_output()
    if (change.new == True):
        with combined_inference_output:
            display_inference_results("combined")

noise_stream.observe(noise_stream_eventhandler, names='value')
noise_volume.observe(noise_volume_eventhandler, names='value')
mammal_volume.observe(mammal_volume_eventhandler, names='value')
mammal_species.observe(mammal_species_eventhandler, names='value')
inf_required.observe(inference_eventhandler, names='value')

### Setup and Launch the Simulation Interface

In [12]:
#Display initial snapshot

species = mammals.get_sample(mammal_species.value)

mammal_processing(species, mammal_volume.value)
noise_processing(noise_stream.value, noise_volume.value)
combined_processing(species,mammal_volume.value,
                        noise_stream.value,noise_volume.value)

mammal_control_widgets = widgets.HBox([mammal_species,mammal_volume])
mammals_tab = widgets.Tab([mammal_plt_output,mammal_audio_output])
mammals_tab.set_title(0,"Mel Spectogram")
mammals_tab.set_title(1,"Audio")
mammal_widgets = widgets.VBox([mammal_control_widgets,mammals_tab])
display(mammal_widgets)

noise_control_widgets = widgets.HBox([noise_stream,noise_volume])
noise_tab = widgets.Tab([noise_plt_output, noise_audio_output])
noise_tab.set_title(0,"Mel Spectogram")
noise_tab.set_title(1,"Audio")
noise_widgets = widgets.VBox([noise_control_widgets, noise_tab])
display(noise_widgets)

combined_tab = widgets.Tab([combined_plt_output,combined_audio_output])
combined_tab.set_title(0,"Mel Spectogram")
combined_tab.set_title(1,"Audio")
display(combined_tab)
display(inf_required)
display(combined_inference_output)

Checkbox(value=False, description='Run Inference')

Output()